## Introduction to B ##

### Basic Datavalues ###

B provides the booleans, strings and integers as built-in datatypes.

In [1]:
BOOL

{FALSE,TRUE}

In [2]:
"this is a string"

"this is a string"

In [3]:
1024

1024

Users can define their own datatype in a B machine.
One distinguishes between explicitly specified enumerated sets and deferred sets.

In [5]:
::load
MACHINE MyBasicSets
SETS Trains = {thomas, gordon}; Points
END

[2018-05-25 13:12:41,869, T+1176691] "Shell-0" de.prob.cli.PrologProcessProvider.makeProcess(PrologProcessProvider.java:64): [INFO] Starting ProB's Prolog Core. Path is /Users/leuschel/git_root/prob_prolog/probcli.sh
[2018-05-25 13:12:43,407, T+1178229] "Shell-0" de.prob.cli.PortPattern.setValue(PortPattern.java:30): [INFO] Server has started and listens on port 54757
[2018-05-25 13:12:43,408, T+1178230] "Shell-0" de.prob.cli.InterruptRefPattern.setValue(InterruptRefPattern.java:29): [INFO] Server can receive user interrupts via reference 94534
[2018-05-25 13:12:43,412, T+1178234] "ProB Output Logger for instance 6e2e7ba7" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] -- starting command loop --
[2018-05-25 13:12:43,441, T+1178263] "ProB Output Logger for instance 6e2e7ba7" de.prob.cli.ProBInstance.readAndLog(ConsoleListener.java:48): [INFO] Connected: 127.0.0.1


Loaded machine: MyBasicSets : []


For animation and constraint solving purposes, ProB will instantiate deferred sets to some finite set (the size of which can be controlled).

In [6]:
Points

{Points1,Points2}

### Pairs ###
B also has pairs of values, which can be written in two ways:

In [7]:
(thomas,10)

(thomas↦10)

In [8]:
thomas |-> 10

(thomas↦10)

Tuples simply correspond to nested pairs:

In [12]:
(thomas |-> gordon |-> 20)

((thomas↦gordon)↦20)

### Sets ###
Sets in B can be specified in multiple ways.
For example, using explicit enumeration:

In [13]:
{1,3,2,3}

{1,2,3}

or via a predicate by using a set comprehension:

In [14]:
{x|x>0 & x<4}

{1,2,3}

For integers there are a variety of other sets, such as intervals:

In [15]:
1..3

{1,2,3}

or the set of implementable integers INT = MININT..MAXINT or the set of implementable natural numbers NAT = 0..MAXINT.

Sets can be higher-order and contain other sets:

In [17]:
{ 1..3,  {1,2,3,2}, 0..1, {x|x>0 & x<4} }

{{0,1},{1,2,3}}

Relations are modelled as sets of pairs:

In [18]:
{ thomas|->gordon, gordon|->gordon, thomas|->thomas}

{(thomas↦thomas),(thomas↦gordon),(gordon↦gordon)}

Functions are relations which map every domain element to at most one value:

In [22]:
{ thomas|->1, gordon|->2}

{(thomas↦1),(gordon↦2)}

## Expressions vs Predicates vs Substitutions ##


### Expressions ###
Expressions in B have a value. With ProB and with ProB's Jupyter backend, you can evaluate expresssions such as:

In [19]:
2**1000

10715086071862673209484250490600018105614048117055336074437503883703510511249361224931983788156958581275946729175531468251871452856923140435984577574698574803934567774824230985421074605062371141877954182153046474983581941267398767559165543946077062914571196477686542167660429831652624386837205668069376

B provides many operators which return values, such as the usual arithmetic operators but also many operators for sets, relations and functions.

In [21]:
(1..3 \/ 5..10) \ (2..6)

{1,7,8,9,10}

In [23]:
ran({(thomas↦1),(gordon↦2)})

{1,2}

In [24]:
{(thomas↦1),(gordon↦2)} (thomas)

1

In [25]:
{(thomas↦1),(gordon↦2)}~[2..3]

{gordon}

## Predicates
ProB can also be used to evaluate predicates (B distinguishes between expressions which have a value and predicates which are either true or false).

In [26]:
2>3

FALSE

In [27]:
3>2

TRUE

Within predicates you can use **open** variables, which are implicitly existentially quantified.
ProB will display the solution for the open variables, if possible.

In [28]:
x*x=100

TRUE

Solution:
	x = −10

We can find all solutions to a predicate by using the set comprehension notation.
Note that by this we turn a predicate into an expression.

In [29]:
{x|x*x=100}

{−10,10}

### Substitutions ###
B also has a rich syntax for substitutions, aka statements.
For example ```x := x+1``` increments the value of x by 1.
We will not talk about substitutions in the rest of this presentation.

## Constraint Solving ##

Constraint solving is determine whether a predicate with open/existentially quantified variables is satisfiable and providing values for the open variables in case it is.
We have already solved the predicate ```x*x=100``` above, yielding the solution ```x=-10```.
The following is an unsatisfiable predicate:

In [31]:
x*x=1000

FALSE

# Constraint solving has many applications in formal methods in general and B in particular.
It is required to animate implicit specifications.
Take for example an event
```
train_catches_up = any t1,t2,x where t1:dom(train_position) & t2:dom(train_position) &
                                     train_position(t1) < train_position(t2) &
                                     x:1..(train_position(t2)-train_position(t1)-1) then
                         train_position(t1) := train_position(t1)+x end
```
To determine whether the event is enabled, and to obtain values for the parameters of the event in a given state of the model, we have to solve the following constraint:

In [39]:
train_position = {thomas|->100, gordon|->2020} &
t1:dom(train_position) & t2:dom(train_position) & train_position(t1) < train_position(t2) &
x:1..(train_position(t2)-train_position(t1)-1)

TRUE

Solution:
	x = 1
	train_position = {(thomas↦100),(gordon↦2020)}
	t1 = thomas
	t2 = gordon

In [40]:
train_position = {thomas|->2019, gordon|->2020} &
t1:dom(train_position) & t2:dom(train_position) & train_position(t1) < train_position(t2) &
x:1..(train_position(t2)-train_position(t1)-1)

FALSE

Suppose that we have the invariant, ```train_position:TRAINS-->1..10000``` we can check whether the event is feasible in at least one valid state by solving:

In [38]:
train_position:Trains-->1..10000 &
t1:dom(train_position) & t2:dom(train_position) & train_position(t1) < train_position(t2) &
x:1..(train_position(t2)-train_position(t1)-1)

TRUE

Solution:
	x = 1
	train_position = {(thomas↦1),(gordon↦3)}
	t1 = thomas
	t2 = gordon

Many other applications exist: generating testcases, finding counter examples using bounded model checking or other algorithms like IC3.
Other applications are analysing proof obligations.
Take the proof obligation for an event theorem t1 $/=$ t2:
```
train_position:Trains-->1..10000 & train_position(t1) < train_position(t2) |- t1 /= t2
```
We can find counter examples to it by negating the proof goal:

In [42]:
train_position:Trains-->1..10000 & train_position(t1) < train_position(t2) & not( t1 /= t2 )

FALSE

Obviously, we can also use constraint solving to solve puzzles or real-life problems.
#### Send More Money Puzzle ####
We now try and solve the SEND+MORE=MONEY arithmetic puzzle in B, involving 8 distinct digits:

In [43]:
{S,E,N,D, M,O,R, Y} <: 0..9 & S >0 & M >0 & 
   card({S,E,N,D, M,O,R, Y}) = 8 & 
   S*1000 + E*100 + N*10 + D +
   M*1000 + O*100 + R*10 + E =
  M*10000 + O*1000 + N*100 + E*10 + Y

TRUE

Solution:
	R = 8
	S = 9
	D = 7
	E = 5
	Y = 2
	M = 1
	N = 6
	O = 0

We can find all solutions (to the unmodified puzzle) using a set comprehension and make sure that there is just a single soltuion:

In [44]:
  {S,E,N,D, M,O,R, Y |
   {S,E,N,D, M,O,R, Y} <: 0..9 &  S >0 & M >0 & 
   card({S,E,N,D, M,O,R, Y}) = 8 & 
   S*1000 + E*100 + N*10 + D +
   M*1000 + O*100 + R*10 + E =
   M*10000 + O*1000 + N*100 + E*10 + Y }

{(((((((9↦5)↦6)↦7)↦1)↦0)↦8)↦2)}

#### KISS PASSION Puzzle####
A slightly more complicated puzzle (involving multiplication) is the KISS * KISS = PASSION problem.

In [46]:
   {K,P} <: 1..9 &
    {I,S,A,O,N} <: 0..9 &
    (1000*K+100*I+10*S+S) * (1000*K+100*I+10*S+S) 
     =  1000000*P+100000*A+10000*S+1000*S+100*I+10*O+N &
    card({K, I, S, P, A, O, N}) = 7

TRUE

Solution:
	P = 4
	A = 1
	S = 3
	I = 0
	K = 2
	N = 9
	O = 8

Finally, a simple puzzle involving sets is to find a subset of numbers from 1..5 whose sum is 14:

In [48]:
x <: 1..5 & SIGMA(y).(y:x|y)=14

TRUE

Solution:
	x = {2,3,4,5}

## How to solve (set) constraints in B ##

### Booleans ###

If we have only booleans, constraint solving is equivalent to SAT solving.
Internally, ProB has an interpreter which does not translate to CNF (conjunctive normal form), but is otherwise similar to DPLL: deterministic propagations are carried out first (unit propagation) and there are heuristics to choose the next boolean variable to enumerate.

#### Knights and Knave Puzzle####
Here is a puzzle from Smullyan involving an island with only knights and knaves.
We know that:
 - Knights: always tell the truth
 - Knaves: always lie

We are given the following information about three persons A,B,C on the island:
 1. A says: “B is a knave or C is a knave”
 2. B says “A is a knight”

What are A, B and C?
Note: we model A,B,C as boolean variables which are equal to TRUE if they are a knight and FALSE if they are a knave.

In [49]:
 (A=TRUE <=> (B=FALSE or C=FALSE)) & // Sentence 1
 (B=TRUE <=> A=TRUE) // Sentence 2

TRUE

Solution:
	A = TRUE
	B = TRUE
	C = FALSE